In [7]:
def mask(time):   
    s = np.zeros((241, 241))
    for x in range(241):
        for y in range(241):
            if np.isnan(time[x][y]) != False:
                s[x][y] = np.nan
    return s

In [8]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

def dbscan(time):
    
    X = time

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # cluster the data into five clusters
    dbscan = DBSCAN(eps=0.3, min_samples=10).fit(X_scaled)
#     clusters = dbscan.fit_predict(X_scaled)
    labels = dbscan.labels_
    
    time['cluster'] = labels
    time = time[(time['cluster'] >= 0)]
    
    return time

In [9]:
def pre_processing(time):
    np.warnings.filterwarnings('ignore')
    
    ## thereshold value to track  21.8
    threshold = 9.96
    
    rs =  (np.where(time != np.nan) and np.where(time > threshold))
    rs = np.asanyarray(rs)
    pe = pd.DataFrame({'x1':rs[0],'y1':rs[1]})
    
    rain = []
    
    for i,row in pe.iterrows():
        rain.append(time[row.x1][row.y1])
    
    pe['rain'] = rain
    
    return pe

In [10]:
def clusterization(time):
    te = time
    
    if len(te) < 2:
        return None
    
    bandwidth = estimate_bandwidth(te, quantile=0.05, n_samples=None, random_state=0, n_jobs=None)
    
    print(bandwidth)

    if bandwidth > 0:
        ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=True, min_bin_freq=1,
        n_jobs=None, seeds=None)

    ms.fit(te)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    n_clusters_ = len(np.unique(labels))
    te['cluster']=labels

#     clust = pd.DataFrame()
    
#     for i in range(n_clusters_):
#         c2 = te.loc[te['cluster'] == i]

#         c3 = ms2(c2[['x1','y1']])
# #         c3.sort_values(by=['time2'], ascending=True)
#         clust = clust.append(c3)
    
#     clust['time1'] = te['cluster']
#     clust = clust.reset_index(drop=True)
#     clust.sort_values(by=['time1', 'time2'], inplace=True)

#     clust['cluster'] = clust.time1.astype(str) + clust.time2.astype(str)
#     clust['cluster'].astype(int)
    
    return te

In [11]:
def ms2(time2):
    bandwidth = estimate_bandwidth(time2, quantile=0.5, n_samples=None)
    
    ms = MeanShift(bandwidth=5, bin_seeding=True, cluster_all=True, min_bin_freq=1,
    n_jobs=None, seeds=None)

    ms.fit(time2)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    n_clusters_ = len(np.unique(labels))
    time2['time2']=labels

    return time2

In [12]:
def defineNewLabels(df):
    def copy(ls):
        return np.array([l for l in ls])

    maxi = df['time1'].max()
    label = 0
    labels = []
    it = 0
    for index in range(maxi+1):
        times = df[df['time1'] == index]
        t1 = times['time1'].values
        t2 = times['time2'].values
        aux = copy(t1)-it
        start = 0

        inc = 0
        temp = set()
        for x in range(len(t2)):
            if start != t2[x]:
                label += 1
                start += 1
            labels.append(label)
        label += 1
        it += 1
    labels

    df['cluster'] = labels
    df
    return df